In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load data from Excel file
file_path = r'C:\Users\christian_luna\Documents\Maestria\TopicosDeIndustria1\datos_trafico_guadalajara.xlsx'
df = pd.read_excel(file_path)

# Clean column names to remove any extra spaces
df.columns = df.columns.str.strip()

# Initialize the Dash application
app = dash.Dash(__name__)

# Filter options
zone_options = [{'label': zone, 'value': zone} for zone in df['Zone'].unique()]
vehicle_options = [{'label': vehicle, 'value': vehicle} for vehicle in df['vehicle type'].unique()]

# Dashboard design
app.layout = html.Div([
    html.H1("Traffic Dashboard - Guadalajara Metropolitan Area"),
    
    # Filters menu
    html.Div([
        html.Label('Select Zone:'),
        dcc.Dropdown(id='zone-dropdown', options=zone_options, value=zone_options[0]['value']),
        
        html.Label('Select Vehicle Type:'),
        dcc.Dropdown(id='vehicle-dropdown', options=vehicle_options, value=vehicle_options[0]['value']),
    ], style={'width': '48%', 'display': 'inline-block'}),
    
    # Graphs
    dcc.Graph(id='heatmap'),
    dcc.Graph(id='speed-histogram')
])

# Callback to update heatmap and histogram based on selected filters
@app.callback(
    [Output('heatmap', 'figure'),
     Output('speed-histogram', 'figure')],
    [Input('zone-dropdown', 'value'),
     Input('vehicle-dropdown', 'value')]
)
def update_graphs(zone, vehicle):
    # Filter the dataset according to the selected zone and vehicle type
    filtered_df = df[(df['Zone'] == zone) & (df['vehicle type'] == vehicle)]
    
    # Heatmap: Congestion versus time
    heatmap_fig = px.density_heatmap(filtered_df, x='Date and Time', y='Zone',
                                     z='numbers of vehicle',
                                     title=f'Congestion in {zone} for {vehicle}',
                                     labels={'z': 'Number of Vehicles'})
    
    # Speed histogram
    speed_histogram_fig = px.histogram(filtered_df, x='Average Speed (km/h)',
                                       title=f'Speed Histogram for {zone} and {vehicle}',
                                       labels={'x': 'Average Speed (km/h)', 'y': 'Frequency'})

    return heatmap_fig, speed_histogram_fig

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)
